https://python.langchain.com/docs/use_cases/graph/graph_cypher_qa

https://python.langchain.com/docs/use_cases/graph/

In [2]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI


In [3]:
graph = Neo4jGraph(url="bolt://3.238.143.207:7687", username="neo4j", password="hulls-fruits-stairs")


In [4]:
print(graph.schema)


Node properties are the following:
Person {name: STRING, born: INTEGER},Movie {tagline: STRING, title: STRING, released: INTEGER}
Relationship properties are the following:
ACTED_IN {roles: LIST},REVIEWED {summary: STRING, rating: INTEGER}
The relationships are the following:
(:Person)-[:ACTED_IN]->(:Movie),(:Person)-[:DIRECTED]->(:Movie),(:Person)-[:PRODUCED]->(:Movie),(:Person)-[:WROTE]->(:Movie),(:Person)-[:FOLLOWS]->(:Person),(:Person)-[:REVIEWED]->(:Movie)


In [5]:
from langchain.llms import Ollama

llm = Ollama(model="openhermes")


In [6]:
chain = GraphCypherQAChain.from_llm(
                                    llm, 
                                    graph=graph, 
                                    verbose=True
)


In [9]:
chain.run("Who played in Hoffa?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title:"Hoffa"}) RETURN p.name AS person, m.title AS movie;
Full Context:
[{'person': 'John C. Reilly', 'movie': 'Hoffa'}, {'person': 'Danny DeVito', 'movie': 'Hoffa'}, {'person': 'J.T. Walsh', 'movie': 'Hoffa'}, {'person': 'Jack Nicholson', 'movie': 'Hoffa'}]

> Finished chain.


'John C. Reilly, Danny DeVito, J.T. Walsh, and Jack Nicholson played in Hoffa.'

In [11]:
movie = "A Few Good Men"

chain.run(f"Who played in {movie}?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (movie:Movie {title: "A Few Good Men"})-[:ACTED_IN]-(person) RETURN person.name AS Person, movie.title AS Movie;
Full Context:
[{'Person': 'James Marshall', 'Movie': 'A Few Good Men'}, {'Person': 'Kevin Pollak', 'Movie': 'A Few Good Men'}, {'Person': 'J.T. Walsh', 'Movie': 'A Few Good Men'}, {'Person': 'Aaron Sorkin', 'Movie': 'A Few Good Men'}, {'Person': 'Cuba Gooding Jr.', 'Movie': 'A Few Good Men'}, {'Person': 'Christopher Guest', 'Movie': 'A Few Good Men'}, {'Person': 'Noah Wyle', 'Movie': 'A Few Good Men'}, {'Person': 'Kiefer Sutherland', 'Movie': 'A Few Good Men'}, {'Person': 'Kevin Bacon', 'Movie': 'A Few Good Men'}, {'Person': 'Demi Moore', 'Movie': 'A Few Good Men'}]

> Finished chain.


'James Marshall, Kevin Pollak, J.T. Walsh, Aaron Sorkin, Cuba Gooding Jr., Christopher Guest, Noah Wyle, Kiefer Sutherland, Kevin Bacon, and Demi Moore played in A Few Good Men.'

In [14]:
movie = "Johnny Mnemonic"

chain.run(f"Who directed the movie {movie}?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie) WHERE m.title = "Johnny Mnemonic" RETURN p.name AS director, m.title AS movie;
Full Context:
[{'director': 'Robert Longo', 'movie': 'Johnny Mnemonic'}]

> Finished chain.


'Robert Longo directed the movie Johnny Mnemonic.'

In [15]:
movie = "Ninja Assassin"

chain.run(f"Who produced the movie {movie}?")




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:PRODUCED]->(m:Movie) WHERE m.title = "Ninja Assassin" RETURN p.name, p.born AS age;
Full Context:
[{'p.name': 'Lana Wachowski', 'age': 1965}, {'p.name': 'Joel Silver', 'age': 1952}, {'p.name': 'Lilly Wachowski', 'age': 1967}]

> Finished chain.


'The movie Ninja Assassin was produced by Joel Silver and Lana Wachowski, along with Lilly Wachowski. The individuals are listed with their respective ages for reference: Lana Wachowski (born in 1965), Joel Silver (born in 1952), and Lilly Wachowski (born in 1967).'